### Find change in OW for different cases

In [61]:
import pandas as pd

CASES = ["Base"] + [f"OnshrWind/OnshrWind{'P' if p > 0 else 'N'}{str(abs(p))}" for p in range(-30, 40, 10) if p != 0]
DF_BY_CASES = dict({})
print(CASES)

['Base', 'OnshrWind/OnshrWindN30', 'OnshrWind/OnshrWindN20', 'OnshrWind/OnshrWindN10', 'OnshrWind/OnshrWindP10', 'OnshrWind/OnshrWindP20', 'OnshrWind/OnshrWindP30']


In [62]:
max_capacities = pd.read_excel("../EMPIRE_extension/EMPIRE_input/Generator.xlsx", sheet_name="MaxInstalledCapacity", header=2)\
                        [["Node", "GeneratorTechnology", "generatorMaxInstallCapacity in MW"]]
max_capacities = max_capacities[max_capacities["GeneratorTechnology"] == "Wind_onshr"]
max_capacities["Node"] = max_capacities["Node"].apply(lambda n: n.replace(" ", ""))
max_capacities

,Node,GeneratorTechnology,generatorMaxInstallCapacity in MW
377,Austria,Wind_onshr,11000.0
378,BosniaH,Wind_onshr,17850.0
379,Belgium,Wind_onshr,8000.0
380,Bulgaria,Wind_onshr,53000.0
381,Switzerland,Wind_onshr,2000.0
382,CzechR,Wind_onshr,76000.0
383,Germany,Wind_onshr,107000.0
384,Denmark,Wind_onshr,55000.0
385,Estonia,Wind_onshr,27000.0
386,Spain,Wind_onshr,704000.0


Installed capacity for cases

In [63]:
MULTIPLY = {
    "Base": 1, 
    "OnshrWind/OnshrWindP10": 1.1, 
    "OnshrWind/OnshrWindP20": 1.2,
    "OnshrWind/OnshrWindP30": 1.3,
    "OnshrWind/OnshrWindN10": 0.9,
    "OnshrWind/OnshrWindN20": 0.8,
    "OnshrWind/OnshrWindN30": 0.7,
}

for case in CASES:
    _df = pd.read_csv(f'Results/{case}/results_output_gen.csv')

    # Onshore wind technology
    _df = _df[_df["GeneratorType"] == "Windonshore"]
    _df = _df.merge(max_capacities, on=["Node"], how="left")
    _df["% of base case max"] = _df.apply(lambda row: round(row["genInstalledCap_MW"] / row["generatorMaxInstallCapacity in MW"] * 100), axis=1)
    _df["generatorMaxInstallCapacity in MW"] = MULTIPLY[case] * _df["generatorMaxInstallCapacity in MW"] if case in MULTIPLY.keys() else _df["generatorMaxInstallCapacity in MW"]
    _df["% of max"] = _df.apply(lambda row: round(row["genInstalledCap_MW"] / row["generatorMaxInstallCapacity in MW"] * 100), axis=1)
    _df["Installed cap GW"] = round(_df["genInstalledCap_MW"] / 1000)
    _df["Max cap GW"] = round(_df["generatorMaxInstallCapacity in MW"] / 1000)
    DF_BY_CASES[case] = _df

DF_BY_CASES

{'Base':         Node GeneratorType     Period  genInvCap_MW  genInstalledCap_MW  \
 0    Austria   Windonshore  2020-2025  1.227146e-06         3500.000001   
 1    Austria   Windonshore  2025-2030  7.500000e+03        10999.999992   
 2    Austria   Windonshore  2030-2035  1.006426e-06        10999.999993   
 3    Austria   Windonshore  2035-2040  1.997185e-06        10999.999994   
 4    Austria   Windonshore  2040-2045  2.543868e-07        10999.999994   
 ..       ...           ...        ...           ...                 ...   
 275      NO5   Windonshore  2035-2040  3.715788e+02         5879.999940   
 276      NO5   Windonshore  2040-2045  4.227700e-05         5879.999982   
 277      NO5   Windonshore  2045-2050  3.618710e-04         5879.999981   
 278      NO5   Windonshore  2050-2055  2.078419e+03         2449.997896   
 279      NO5   Windonshore  2055-2060  8.017330e-06         2449.997797   
 
      genExpectedCapacityFactor  DiscountedInvestmentCost_Euro  \
 0          

In [1]:
FILTER_PERIOD = "2045-2050" # Installed capacity at end of period (e.g. 2045-2050 = 2050)
FILTER_CASE = "Base" # Base, P10, N10 ...

df_filtered = DF_BY_CASES[f"OnshrWind/OnshrWind{FILTER_CASE}" if FILTER_CASE != "Base" else "Base"]
df_filtered = df_filtered[(df_filtered["Period"] == FILTER_PERIOD)][["Node", "GeneratorType", "Period", "Installed cap GW", "Max cap GW", "% of max", "% of base case max"]]
print(sum(df_filtered["Installed cap GW"]))
df_filtered.sort_values(by=["% of max", "Installed cap GW"], ascending=False)

NameError: name 'DF_BY_CASES' is not defined